# Temperature Ramps

In [1]:
#-----Import esential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Declare some important variables

t_0 = 5.6;
t_1 = 5.8;
t_2 = 6.0;
t_3 = 6.2;
p_max  = 1.4;
steps_ramp = 5e4;


#-----Crea un contexto de simulación

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Extrae la configuración del centro de masa de las partículas

system = hoomd.init.read_gsd("T_CM_5.6_P_1.4_ramp.gsd",frame = -1)

#-----Define cada mesógeno en el marco de referencia local de cada centro de masa.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types=['A']*8,
                positions=[(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Declara que las moléculas creadas son cuerpos rígidos

rigid.create_bodies();

#-----Define la energía potencial

nl = hoomd.md.nlist.tree();

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl);
lj.set_params(mode = 'shift');

#-----Define la interacción entre las especies de la simulación

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0);

#-----Selecciona un integrador standard

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define el grupo sobre el cual se integraran las ecuaciones de movimiento

mesogens = hoomd.group.rigid_center();

#----Integrate using NPT

temp1 = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);
temp2 = hoomd.variant.linear_interp(points = [(0,t_1), (steps_ramp, t_2)]);
temp3 = hoomd.variant.linear_interp(points = [(0,t_2), (steps_ramp, t_3)]);

npt = hoomd.md.integrate.npt(group = mesogens, kT = temp1, tau = 1.5, tauP = 1.5, P = p_max);
npt.randomize_velocities(seed = 42);

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [2]:
#-----Ramp1 T : 5.6 -> 5.8
#-----Write output and Run the Simulation

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

In [ ]:
hoomd.run(steps_ramp)
system.box.get_volume()

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:43 | Step 1445000 / 1495000 | TPS 0 | ETA -2562047:-47:-16
Time 00:01:25 | Step 1445001 / 1495000 | TPS 0.0237705 | ETA 584:16:49
Time 00:02:05 | Step 1445101 / 1495000 | TPS 2.53643 | ETA 05:27:52
Time 00:02:44 | Step 1445201 / 1495000 | TPS 2.53568 | ETA 05:27:19
Time 00:03:23 | Step 1445301 / 1495000 | TPS 2.54121 | ETA 05:25:57
Time 00:04:03 | Step 1445401 / 1495000 | TPS 2.5465 | ETA 05:24:37
Time 00:04:42 | Step 1445501 / 1495000 | TPS 2.54824 | ETA 05:23:44
Time 00:05:21 | Step 1445601 / 1495000 | TPS 2.54007 | ETA 05:24:07
Time 00:06:01 | Step 1445701 / 1495000 | TPS 2.54005 | ETA 05:23:28
Time 00:06:40 | Step 1445801 / 1495000 | TPS 2.5366 | ETA 05:23:15


In [ ]:
system.get_metadata()

In [ ]:
#-----Ramp2 T : 5.8 -> 6.0
#-----Update parameters

npt.set_params(kT = temp2, tau = 3.0, tauP = 3.0);

#-----Write output and Run the Simulation

log_file = "T_" + str(t_2) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_2) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_2) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

In [ ]:
hoomd.run(steps_ramp)
system.box.get_volume()

In [ ]:
system.get_metadata()

In [ ]:
#-----Ramp2 T : 6.0 -> 6.2
#-----Update parameters

npt.set_params(kT = temp3,tau = 4.0 ,tauP = 4.0);

#-----Write output and Run the Simulation

log_file = "T_" + str(t_3) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_3) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_3) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True);

In [ ]:
hoomd.run(steps_ramp)
system.box.get_volume()

In [ ]:
system.get_metadata()